In [ ]:
# メタデータ登録フォームを表示する
import os
import traceback

import jsonschema
import panel as pn
from IPython.display import display, clear_output
from IPython.core.display import Javascript

from library.task_director import TaskDirector
from library.utils.widgets import Button
from library.utils import dg_web
from library.utils.storage_provider import grdm
from library.utils.input import get_token, get_project_id
from library.utils.file import JsonFile

notebook_name = 'governance_sheet.ipynb'

class Metadata(TaskDirector):

    def __init__(self, working_path:str) -> None:
        super().__init__(working_path, notebook_name)

        self.schema_form = dg_web.Form()

        # フォームボックス
        self._form_box = self.schema_form.form_box
        # メッセージ用ボックス
        self._msg_output = self.schema_form.msg_output

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # get parameters
        self.token = get_token()
        self.project_id = get_project_id()
        self.remote_path = dg_web.GOVSHEET_PATH
        self.base_url = grdm.API_V2_BASE_URL
        clear_output()

        # get datas
        data = grdm.download_json_file(
            token=self.token, base_url=self.base_url,
            project_id=self.project_id, remote_path=self.remote_path
        )
        self.schema = dg_web.get_govsheet_schema(
            scheme=dg_web.SCHEME, domain=dg_web.DOMAIN
        )

        # define a form
        self.schema_form.create_widgets(self.schema, data)
        submit_button = Button(name='保存する', button_type='primary')
        submit_button.on_click(self.submit)
        self._form_box.append(submit_button)

        # display a form
        pn.extension()
        display(self._form_box)
        display(self._msg_output)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    @TaskDirector.callback_form('ガバナンスシートを保存する')
    def submit(self, event):
        file_path = os.path.join(self._abs_root_path, self.remote_path)
        tmpfile = JsonFile(file_path)
        try:
            data = self.schema_form.get_data()
            # validate input data
            jsonschema.validate(data, self.schema)
            # save to remote
            tmpfile.write(data)
            grdm.sync(
                    token=self.token,
                    base_url=self.base_url,
                    project_id=self.project_id,
                    abs_source = file_path,
                    abs_root=self._abs_root_path
                )
        except jsonschema.ValidationError as e:
            self._msg_output.update_warning(e.message)
            self.log.warning(e.message)
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        finally:
            tmpfile.remove(missing_ok=True)

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

notebook_name = 'governance_sheet.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()